<a href="https://colab.research.google.com/github/lanlooker/Looker-Tutorials/blob/main/01_Transfer_All_Schedules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Purpose**: This script transfers all schedules of a user to a different user. The script may come in handy when a user leaves an organization, and Looker admins have to re-assign all of their existing schedules to a new owner. 

Please click "Open in Colab" to run the script as a one-off-time task. You can also download the file as a .py file to run inside a server (refer to [this page ](https://github.com/looker-open-source/sdk-codegen/blob/main/README.md#configuring-lookerini-or-env
) to learn about the init file) 

**How to run as a one-off-time task in Google Colab**: 
1. Edit your client ID and secret key in the first cell 
2. Run the second cell to call the function `update_owner('current_owner@looker.com', 'new_owner@looker.com')`

**Note**: The script is out of scope for Looker chat support. For questions and comments, please post in Looker Community. 



In [ ]:
# Install Looker SDK
!pip install looker_sdk
import looker_sdk
import os

# Login 
os.environ['LOOKERSDK_BASE_URL'] = 'XXX'
os.environ['LOOKERSDK_CLIENT_ID'] = 'YYY'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'ZZZ'

#Initialize the SDK
sdk = looker_sdk.init31()  # or init40() for v4.0 API 

# Return the user id of by looking up email
def find_user_id(email: str):
  user_id = sdk.search_users(email=email)
  if len(user_id) == 0:
    return 'There is no user associated with this email'
  else:
    return user_id[0]['id']
  
# Return all schedules of a particular user id
def find_schedules(user_id: int):
  result = {}
  schedule_plans = sdk.all_scheduled_plans(user_id=user_id)
  for i in schedule_plans:
    result[i['name']] = i['id']
  return result

# Transfer all schedules of a user to a new user.  
# If additional condition is needed, please adjust in the loop for sdk.update_schedule_plan() 

def update_owner(current_owner_email: str, new_owner_email: str):
  current_owner_id = find_user_id(current_owner_email)
  new_owner_id = find_user_id(new_owner_email)  
  
  if type(new_owner_id) != int and type(new_owner_id) != int:
    return "The email addresses for both the current owner and the new owner are not associatd with any Looker user id"

  elif type(current_owner_id) != int: 
    return "The email address for the current owner is not associated with any Looker user id"

  elif type(new_owner_id) != int:
    return "The email address for the new owner is not associated with any Looker user id"

  else: 
    body = {}
    body['user_id'] = new_owner_id
    find = find_schedules(current_owner_id) 
    for i in find.values(): 
        sdk.update_scheduled_plan(i,body)

    if find == find_schedules(new_owner_id) and find_schedules(current_owner_id) == {}: 

      result = "Successfully transfer the following schedules from " + current_owner_email + " to " + new_owner_email
      print(result)
      for i in find: 
        print(i)
      return None
      
    else: 
      return "Something went wrong, please check if email is correct or if the user has any schedules."

In [ ]:
# Uncomment the line and make changes to the parameter:
# update_owner('current_owner@looker.com', 'new_owner@looker.com')